In [41]:
import pandas as pd
import numpy as np
!pip install pandasql
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [129]:
def get_top10(df, factor, quater, asc_list): #get the quater return for top10
    
    if factor in asc_list:
        top10=df[quater].nsmallest(10, factor)
            
    else:
        top10=df[quater].nlargest(10, factor)
        
    return top10

def get_3m_ret(df, quater, top10): #get quater return of given companies
    tickers=list(top10['Ticker'])
    companies=df[quater].loc[df[quater]['Ticker'].str.contains(tickers[0], case=False)]
    for ticker in tickers[1:]:
        temp_company=df[quater].loc[df[quater]['Ticker'].str.contains(ticker, case=False)]
        companies=pd.concat([companies, temp_company])
        #companies=companies.append(df['Q1 2018'].loc[df['Q1 2018']['Ticker'].str.contains(ticker, case=False)])
        
    _3m_ret=[x for x in companies['3M Return%'] if not(pd.isnull(x)) == True]
    #print(_3m_ret)
    avg_3m_ret=sum(_3m_ret)/len(_3m_ret)

    return avg_3m_ret
    
def get_5yr_ret(avg_3m_ret):
    #total return = (1+1stQ_return)*(1+2ndQ_return)...*(1+20thQ_return)-1
    
    total_rt = 1+avg_3m_ret[0]
    #print(total_rt)
    for i in range(1,len(avg_3m_ret)):
        total_rt = (1+avg_3m_ret[i])*total_rt
        #print(total_rt)
    total_rt = (total_rt) - 1
    #print(total_rt)
    return total_rt
        
#select variable and build the porfolio
def variable_select(_5yr_ret,value_fac,growth_fac,profitability_fac,solency_fac,divident_fac,momentum_fac):
    value_max=-1
    growth_max=-1
    profitability_max=-1
    solency_max=-1
    divident_max=-1
    momentum_max=-1
    selected_var = {}
    for key in _5yr_ret:
        if key in value_fac and _5yr_ret[key]>value_max:
            value_max=_5yr_ret[key]
            value_factor=key
            
        elif key in growth_fac and _5yr_ret[key]>growth_max:
            growth_max=_5yr_ret[key]
            growth_factor=key
            
        elif key in profitability_fac and _5yr_ret[key]>profitability_max:
            profitability_max=_5yr_ret[key]
            profitability_factor=key
            
        elif key in solency_fac and _5yr_ret[key]>solency_max:
            solency_max=_5yr_ret[key]
            solency_factor=key
        
        elif key in divident_fac and _5yr_ret[key]>divident_max:
            divident_max=_5yr_ret[key]
            divident_factor=key
        
        elif key in momentum_fac and _5yr_ret[key]>momentum_max:
            momentum_max=_5yr_ret[key] 
            momentum_factor=key
        else:
            pass
    
    selected_var[value_factor]=value_max
    selected_var[growth_factor]=growth_max
    selected_var[profitability_factor]=profitability_max
    selected_var[solency_factor]=solency_max
    selected_var[divident_factor]=divident_max
    selected_var[momentum_factor]=momentum_max

    return selected_var
        
def get_rank(df, variables, quater, asc_list):
    #returns the companies that has smallest sum of rank wrpt selected variables
    for variable in variables:
        #rank compenies based on the porfolio
        new_col = variable + 'rank'
        if variable in asc_list:
            df[quater][new_col] = df[quater][variable].rank(ascending=True)
        else:
            df[quater][new_col] = df[quater][variable].rank(ascending=False)
            
    #sum the rank
    df[quater]['Sum Rank']=df[quater].iloc[:,36:41].sum(axis=1)
    top10 = df[quater].nsmallest(10, 'Sum Rank')
    return top10

In [42]:
quaters = list(df.keys())
factors = df[quaters[0]].columns
#lower-better factors
asc_list = ['TEV/Revenue','TEV/EBITDA','TEV/EBIT','P/S','P/E','P/B','Debt/EBITDA','Debt/Equity','Debt/Capital']
value_fac = ['TEV/Revenue','TEV/EBITDA','TEV/EBIT','P/S','P/E','P/B']
growth_fac = ['Revenue G%','GP G%','EBITDA G%','EBIT G%','NI G%','EPS G%']
profitability_fac = ['Gross M%','EBITDA M%','EBIT M%','NI M%','UFCF M%','LFCF M%']
solency_fac = ['Debt/EBITDA','Debt/Equity','Debt/Capital']
divident_fac = ['Dividend Y%','Dividend G%','Dividend P%']
momentum_fac = ['1M Return%','3M Return%','6M Return%']


In [ ]:
#data cleanning

df=pd.read_excel(r'/Users/zjy/Documents/Factor Model Template.xlsx', skiprows=lambda x: x in [0, 3] , header=3, sheet_name=None, na_values=['#VALUE!','NM',0,'#DIV/0!'])
for key in df:
    sheet = df[key]
    #convert empty cells to NaN
    sheet.drop(sheet.columns[[0]], axis=1, inplace=True) #delete first coln
    

In [134]:
df['Q1 2018'].head()

,Ticker,Company,Sector,Industry,Market Cap,TEV,TEV/Revenue,TEV/EBITDA,TEV/EBIT,P/S,...,Dividend P%,1M Return%,3M Return%,6M Return%,TEV/Revenuerank,GP G%rank,Debt/EBITDArank,Dividend Y%rank,6M Return%rank,Sum Rank
0,AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage and Peripherals",851317.90114,688620.90114,2.87914,9.28386,10.71633,3.62727,...,0.246895,-0.041257,-0.008568,0.090826,171.0,163.0,111.0,186.0,153.0,784.0
1,MSFT,Microsoft Corporation,Information Technology,Systems Software,702760.09360,664396.09360,6.94597,19.07759,27.23046,7.36108,...,0.791653,-0.017017,0.066986,0.223294,348.0,249.0,144.0,162.0,58.0,961.0
2,GOOG,Alphabet Inc.,Communication Services,Interactive Media and Services,719249.21638,621347.21638,5.60504,17.36528,21.51330,23.51781,...,0.000438,-0.035278,-0.013962,0.082369,315.5,54.5,9.5,351.5,168.0,899.0
3,GOOGL,Alphabet Inc.,Communication Services,Interactive Media and Services,719249.21638,621347.21638,5.60504,17.36528,21.51330,6.48266,...,0.000438,-0.031986,-0.015436,0.072013,315.5,54.5,9.5,351.5,179.0,910.0
4,AMZN,"Amazon.com, Inc.",Consumer Discretionary,Internet and Direct Marketing Retail,700667.69024,713828.69024,4.01329,47.47780,174.01967,3.90588,...,0.000438,-0.030875,0.237603,0.508919,249.0,11.0,149.0,351.5,8.0,768.5


In [91]:
df['Q1 2018'].isnull().sum()

Ticker           0
Company          0
Sector           0
Industry         0
Market Cap       0
TEV              0
TEV/Revenue      0
TEV/EBITDA       8
TEV/EBIT        13
P/S              0
P/E             46
P/B             20
Revenue G%       0
GP G%            1
EBITDA G%        9
EBIT G%         17
NI G%           61
EPS G%          59
Gross M%         0
EBITDA M%        0
EBIT M%          1
NI M%            1
UFCF M%          0
LFCF M%          0
ROA%             0
ROE%            14
ROC%             0
Debt/EBITDA      8
Debt/Equity     18
Debt/Capital     0
Dividend Y%      0
Dividend G%      0
Dividend P%     21
1M Return%      12
3M Return%      12
6M Return%      12
dtype: int64

In [106]:
for factor in factors[6:]: #for each column after TEV
    for quater in quaters[:19]: #for each quater
        if factor in asc_list:
            df[quater][factor].fillna(df[quater][factor].max(),inplace=True)
        else:
            df[quater][factor].fillna(df[quater][factor].min(),inplace=True)

In [107]:
df['Q1 2018'].isnull().sum()

Ticker           0
Company          0
Sector           0
Industry         0
Market Cap      12
TEV             12
TEV/Revenue      0
TEV/EBITDA       0
TEV/EBIT         0
P/S              0
P/E              0
P/B              0
Revenue G%       0
GP G%            0
EBITDA G%        0
EBIT G%          0
NI G%            0
EPS G%           0
Gross M%         0
EBITDA M%        0
EBIT M%          0
NI M%            0
UFCF M%          0
LFCF M%          0
ROA%             0
ROE%             0
ROC%             0
Debt/EBITDA      0
Debt/Equity      0
Debt/Capital     0
Dividend Y%      0
Dividend G%      0
Dividend P%      0
1M Return%       0
3M Return%       0
6M Return%       0
dtype: int64

In [121]:
_5yr_ret={}
variables={}
for factor in factors[6:]: #for each column after TEV
    avg_3m_ret=[]
    for quater in quaters[:19]: #for each quater
        top10 = get_top10(df, factor, quater, asc_list) #get a dataframe
        next_qt = quaters[quaters.index(quater) + 1]
        _3m_ret = get_3m_ret(df, next_qt, top10) #average quater return for top10
        avg_3m_ret.append(_3m_ret)
    _5yr_ret[factor]=get_5yr_ret(avg_3m_ret)
    #print(avg_3m_ret)

In [122]:
_5yr_ret 

{'TEV/Revenue': 1.1180681457066122,
 'TEV/EBITDA': 0.4786381804775621,
 'TEV/EBIT': 0.39163591662614716,
 'P/S': 0.8104862655332141,
 'P/E': 0.08211117915303401,
 'P/B': 0.6166938415152594,
 'Revenue G%': 1.5161629687685338,
 'GP G%': 2.6726540324016708,
 'EBITDA G%': 2.24474820841248,
 'EBIT G%': 0.8823442337716774,
 'NI G%': 0.7845936593487914,
 'EPS G%': 0.48416173855215594,
 'Gross M%': 0.4160613799759749,
 'EBITDA M%': 0.36504802827457383,
 'EBIT M%': 0.3352261101024694,
 'NI M%': 0.3339801412728496,
 'UFCF M%': 0.11536281985571528,
 'LFCF M%': 0.1783958707890081,
 'ROA%': 0.23263978684757602,
 'ROE%': 0.4579930283686977,
 'ROC%': 0.18291089337077593,
 'Debt/EBITDA': 0.8304185951737091,
 'Debt/Equity': 0.27815157540248237,
 'Debt/Capital': 0.27815157540248237,
 'Dividend Y%': 1.0861600362169868,
 'Dividend G%': 0.627830804911657,
 'Dividend P%': 0.37126278194955953,
 '1M Return%': 0.2675178130831919,
 '3M Return%': 1.1397210880085593,
 '6M Return%': 3.923640642880809}

In [112]:
df_=pd.read_csv(r'/Users/zjy/Downloads/SPY.csv')
#S&P 500 5yr return
SP_500_yr_ret=df_['Adj Close'][1448]/df_['Adj Close'][126]-1
SP_500_yr_ret

0.7050264967711994

In [123]:
#within each factor, choose the variables that outperform the S&P 500
variables=variable_select(_5yr_ret,value_fac,growth_fac,profitability_fac,solency_fac,divident_fac,momentum_fac)
variables

{'TEV/Revenue': 1.1180681457066122,
 'GP G%': 2.6726540324016708,
 'Gross M%': 0.4160613799759749,
 'Debt/EBITDA': 0.8304185951737091,
 'Dividend Y%': 1.0861600362169868,
 '6M Return%': 3.923640642880809}

In [124]:
del variables['Gross M%']

In [125]:
variables

{'TEV/Revenue': 1.1180681457066122,
 'GP G%': 2.6726540324016708,
 'Debt/EBITDA': 0.8304185951737091,
 'Dividend Y%': 1.0861600362169868,
 '6M Return%': 3.923640642880809}

In [130]:
new_top10 = get_rank(df, variables, 'Q4 2017', asc_list)
next_qt = quaters[quaters.index(quater) + 1]
_3m_ret = get_3m_ret(df, next_qt, new_top10)
new_top10

,Ticker,Company,Sector,Industry,Market Cap,TEV,TEV/Revenue,TEV/EBITDA,TEV/EBIT,P/S,...,Dividend P%,1M Return%,3M Return%,6M Return%,TEV/Revenuerank,GP G%rank,Debt/EBITDArank,Dividend Y%rank,6M Return%rank,Sum Rank
238,TSCO,Tractor Supply Company,Consumer Discretionary,Specialty Stores,9385.93621,9862.47821,1.36598,11.43716,14.09014,1.33532,...,0.316678,0.096684,0.186696,0.360080,54.0,114.0,24.0,187.0,32.0,411.0
68,BA,The Boeing Company,Industrials,Aerospace and Defense,175642.06222,176437.06222,1.93229,15.74066,19.22810,1.96951,...,0.403996,0.086705,0.152037,0.485019,91.0,197.0,39.0,106.0,14.0,447.0
233,TSN,"Tyson Foods, Inc.",Consumer Staples,Packaged Foods and Meats,29782.59332,39682.59332,1.03718,10.04368,12.25906,0.77553,...,0.123755,-0.016021,0.141992,0.295462,35.0,50.0,158.0,182.0,48.0,473.0
229,DHI,"D.R. Horton, Inc.",Consumer Discretionary,Homebuilding,19202.46856,21101.56856,1.49752,12.51576,12.93543,1.35658,...,0.156054,0.012691,0.277389,0.486321,62.0,22.0,157.0,233.0,13.0,487.0
378,PHM,"PulteGroup, Inc.",Consumer Discretionary,Homebuilding,9774.42430,13070.58730,1.57995,11.42465,11.97389,1.26022,...,0.252108,-0.020330,0.214834,0.357697,72.0,43.0,124.0,220.0,35.0,494.0
106,LRCX,Lam Research Corporation,Information Technology,Semiconductor Equipment,29809.44587,26073.20387,2.94302,10.02549,11.38962,3.37844,...,0.174157,-0.019914,-0.010110,0.321488,173.0,19.0,48.0,219.0,42.0,501.0
218,LYB,LyondellBasell Industries N.V.,Materials,Commodity Chemicals,43516.85831,49951.85831,1.50930,7.59262,9.10699,1.33917,...,0.290018,0.059750,0.097166,0.292409,64.0,277.0,71.0,40.0,52.0,504.0
196,ROST,"Ross Stores, Inc.",Consumer Discretionary,Apparel Retail,30667.21626,29919.37726,2.20367,13.36340,15.47604,2.27139,...,0.197550,0.059686,0.240532,0.378156,116.0,96.0,13.0,251.0,30.0,506.0
104,MU,"Micron Technology, Inc.",Information Technology,Semiconductors,47547.67165,50989.67165,2.20210,3.98201,5.91185,1.97564,...,0.001100,-0.020719,0.027743,0.411119,115.0,3.0,26.0,349.5,20.0,513.5
124,VLO,Valero Energy Corporation,Energy,Oil and Gas Refining and Marketing,40218.10252,44381.10252,0.53492,8.25849,13.01499,0.49324,...,0.305535,0.091957,0.192862,0.351220,10.0,343.0,103.0,53.0,38.0,547.0


In [131]:
top10_3m_ret=[]
for quater in quaters[:19]:
    #rank compenies based on the porfolio, sum the rank, choose the top 10 with lowest average ranking
    #calculate the return          
    new_top10 = get_rank(df, variables, quater, asc_list)
    next_qt = quaters[quaters.index(quater) + 1]
    _3m_ret = get_3m_ret(df, next_qt, new_top10)
    top10_3m_ret.append(_3m_ret)

In [132]:
_5yr_ret=get_5yr_ret(top10_3m_ret)      

In [133]:
_5yr_ret

0.9331333441371574